## **12. k-Nearest Neighbors**

In [2]:
from collections import Counter
from linear_algebra import distance
from stats import mean
import math, random
import matplotlib.pyplot as plt

### **Nearest neighbors classification**
Imagine that you’re trying to predict how I’m going to vote in the next presidential election.
- one sensible approach is to look at how my neighbors are planning to vote.

Now imagine you know more about me than just geography — perhaps you know **my age, my income, how
many kids I have, and so on.**
- Look at my neighbors who are close to me among all those dimensions

### **Very simple Model**
- Some notion of distance
- An assumption that points that are close to one another are similar
- Find the k nearest labeled points and let them vote on the new output.
    - The labels could be True and False (“is spam?” or “is poisonous?” or “would be enjoyable to watch?”)
    - Or they could be categories, like movie ratings (G, PG, PG-13, R, NC-17).
    - Or they could be the names of presidential candidates.
    - Or they could be favorite programming languages

<img src = "picture/le12-1.png">

The following raw_majority_vote does not consider ties.

In [5]:
def raw_majority_vote(labels):
    votes = Counter(labels)
    winner, _ = votes.most_common(1)[0]
    return winner

**Tie-Breakers**
- Pick one of the winners at random.
- Weight the votes by distance and pick the weighted winner.
- Reduce k until we find a unique winner.
    - This is used in the following implementation.

In [7]:
def majority_vote(labels):
    """assumes that labels are ordered from nearest to farthest"""
    vote_counts = Counter(labels)
    winner, winner_count = vote_counts.most_common(1)[0]
    num_winners = len([count
                       for count in vote_counts.values()
                       if count == winner_count])
    if num_winners == 1:
        return winner # unique winner, so return it
    else:
        return majority_vote(labels[:-1]) # try again without the farthest

Note on the above implementation: If the farthest is more than one, they should be removed all at once. In this
case, there might be no decision!

The following knn_classify implementation is partially wrong, too. It doesn't consider k-nearest in tie.

In [9]:
def knn_classify(k, labeled_points, new_point):
    """each labeled point should be a pair (point, label)"""
    
    # order the labeled points from nearest to farthest
    by_distance = sorted(labeled_points,
                         key=lambda point_label: distance(point_label[0], new_point))
    
    # find the labels for the k closest
    k_nearest_labels = [label for _, label in by_distance[:k]]
    
    # and let them vote
    return majority_vote(k_nearest_labels)

## **Example: Favorite Languages**
The VP of Community Engagement wants to know if we can use these results to predict the favorite
programming languages for places that weren’t part of our survey

```python
cities = [(-86.75,33.5666666666667,'Python'),(-88.25,30.6833333333333,'Python'),(-112.01666
cities = [([longitude, latitude], language) for longitude, latitude, language in cities]
```

**Terminology:**
- k is a **hyperparameter**, which we choose and experiment with.
- Correctness based on **"Leave-one-out cross validation"** (one is test data, and the others are training
data)

```python
# try several different values for k
for k in [1, 3, 5, 7]:
    num_correct = 0
    
    for location, actual_language in cities:
        other_cities = [other_city
                        for other_city in cities
                        if other_city != (location, actual_language)]
        
    predicted_language = knn_classify(k, other_cities, location)
    
    if predicted_language == actual_language:
        num_correct += 1
        
    print(k, "neighbor[s]:", num_correct, "correct out of", len(cities))
``` 
1 neighbor[s]: 40 correct out of 75  
3 neighbor[s]: 44 correct out of 75  
5 neighbor[s]: 41 correct out of 75  
7 neighbor[s]: 35 correct out of 75      

It looks like 3-nearest neighbors performs the best, giving the correct result about 59% of the time

In [1]:
def plot_state_borders(plt, color='0.8'):
    pass
def plot_cities():
    # key is language, value is pair (longitudes, latitudes)
    plots = { "Java" : ([], []), "Python" : ([], []), "R" : ([], []) }
    
    # we want each language to have a different marker and color
    markers = { "Java" : "o", "Python" : "s", "R" : "^" }
    colors = { "Java" : "r", "Python" : "b", "R" : "g" }
    
    for (longitude, latitude), language in cities:
        plots[language][0].append(longitude)
        plots[language][1].append(latitude)
        
    # create a scatter series for each language
    for language, (x, y) in plots.items():
        plt.scatter(x, y, color=colors[language], marker=markers[language],
                    label=language, zorder=10)
        
    plot_state_borders(plt) # assume we have a function that does this
    
    plt.legend(loc=0) # let matplotlib choose the location
    plt.axis([-130,-60,20,55]) # set the axes
    plt.title("Favorite Programming Languages")
    plt.show()

```
plot_cities()
```
<img src = "picture/le12-2.png">

### **Classification regions**
Look at what regions would get classified to which languages under each nearest neighbors scheme.

In [2]:
def classify_and_plot_grid(k=1):
    plots = { "Java" : ([], []), "Python" : ([], []), "R" : ([], []) }
    markers = { "Java" : "o", "Python" : "s", "R" : "^" }
    colors = { "Java" : "r", "Python" : "b", "R" : "g" }

    for longitude in range(-130, -60):
        for latitude in range(20, 55):
            predicted_language = knn_classify(k, cities, [longitude, latitude])
            plots[predicted_language][0].append(longitude)
            plots[predicted_language][1].append(latitude)

        # create a scatter series for each language
        for language, (x, y) in plots.items():
            plt.scatter(x, y, color=colors[language], marker=markers[language],
                        label=language, zorder=0)

    plot_state_borders(plt, color='black') # assume we have a function that does this

    plt.legend(loc=0) # let matplotlib choose the location
    plt.axis([-130,-60,20,55]) # set the axes
    plt.title(str(k) + "-Nearest Neighbor Programming Languages")
    plt.show()

In [3]:
classify_and_plot_grid()

NameError: name 'knn_classify' is not defined

In [4]:
classify_and_plot_grid(3)

NameError: name 'knn_classify' is not defined

### **The Curse of Dimensionality**
- k-nearest neighbors runs into trouble in higher dimensions thanks to the “curse of dimensionality,” which
boils down to the fact that high-dimensional spaces are vast.
- Points in high-dimensional spaces tend not to be close to one another at all.
- One way to see this is by randomly generating pairs of points in the d-dimensional “unit cube” in a variety
of dimensions, and calculating the distances between them.

### **Throw 50 random numbers in 1, 2, 3 dimensions**
<img src = "picture/le12-3.png">

In [24]:
def random_point(dim):
    return [random.random() for _ in range(dim)]

def random_distances(dim, num_pairs):
    return [distance(random_point(dim), random_point(dim))
            for _ in range(num_pairs)]

In [26]:
dimensions = range(1, 101, 5)

avg_distances = []
min_distances = []

random.seed(0)
for dim in dimensions:
    distances = random_distances(dim, 10000) # 10,000 random pairs
    avg_distances.append(mean(distances)) # track the average
    min_distances.append(min(distances)) # track the minimum
    print(dim, min(distances), mean(distances), min(distances) / mean(distances))

1 7.947421226228712e-06 0.3310009902894413 2.4010264196729895e-05
6 0.18647467260473205 0.9677679968196268 0.19268530600055306
11 0.315888574043911 1.3334395796543002 0.23689755341281116
16 0.7209190490469604 1.6154152410436047 0.4462747600308797
21 0.9694045860570238 1.8574960773724116 0.5218878240800003
26 1.1698067560262715 2.0632214700056446 0.5669807013122402
31 1.2930748713962408 2.257299829279505 0.5728414340991512
36 1.5123637311959328 2.437670913316559 0.620413413038717
41 1.5514668006745476 2.6039686964057926 0.5958085451703037
46 1.6688006850159558 2.756796053135482 0.6053406392242623
51 2.0135369208019926 2.902997336534375 0.6936061895274667
56 2.1422705294432887 3.0461953095695335 0.7032610557548324
61 2.2891825062886793 3.1783717877656223 0.720237486092828
66 2.3805561409678484 3.305579571524835 0.7201630121006946
71 2.428355816745725 3.4329484139337785 0.7073674066552892
76 2.5356413086431617 3.558475062222762 0.7125640237195596
81 2.682272988673655 3.669873368578009 0.7

#### **Throw 10,000 distances for every dimension from 1 to 100**
<img src = "picture/le12-4.png">

**Note on indexing:**
- For millions of data, we must index them to search efficiently.
- Without indexing, learning take no time, prediction takes a lot of time!!!

### **K-d Tree for Indexing**
Step 1. for each dimension d in {1, ... , k}:
    find a median on the dimension d
    split data based on a median
Step 2. repeat Step 1 until no data is found  
<img src = "picture/le12-5.png">
- You may use **a even splitter** instead of median if you want to leave **all the data points in leaf** (This is
usually used in DB applications)
- Note that the above diagram is equivalent to a balanced k-dimensional BST
- We will not go deep in detail
- However, you can think about range query, k nearest neighbor query
- When k is large, we experience the curse of dimensionality; That is, indexing for large k is almost
impossible!

#### **help(neighbors.KNeighborsClassifier)**
- Take a look at the parameters: algorithm, p, metric

In [29]:
help(neighbors.KNeighborsClassifier)

NameError: name 'neighbors' is not defined

#### **help(neighbors.KDTree)**
    tree.query(...) # knn query  
    tree.query_radius(...) # range query

- If you use KDTree in your real application, construct it and then pickle it (googling for python pickle and
unpickle) (or you can use a DBMS suppoting GIS functionality)
- Your google application may be:
    - Find k nearest gas-stations from me : knn query
    - Find restaurants within 5 minute walking distance : range query

In [31]:
from sklearn.neighbors import KDTree
import numpy as np
np.random.seed(0)
X = np.random.random((10000000, 2)) # 10 million points in 2 dimensions
tree = KDTree(X, leaf_size=1)

In [ ]:
%time dist, ind = tree.query([X[0]], k=5)
print(ind) # indices of 3 closest neighbors
print(dist) # distances to 3 closest neighbors

#### **Compare to Homework #2's result**
CPU times: user 401 ms, sys: 264 ms, total: 665 ms

In [32]:
count = tree.query_radius([X[0]], r = 0.0005, count_only=True)
print(count)

[9]


In [33]:
print(X[tree.query_radius([X[0]], r = 0.0005)[0]])

[[0.5486868  0.71490509]
 [0.54875985 0.71498733]
 [0.54850997 0.71539435]
 [0.5488135  0.71518937]
 [0.54892172 0.71510044]
 [0.54897222 0.71522494]
 [0.54902884 0.71487778]
 [0.54891016 0.7152985 ]
 [0.54892252 0.71534665]]


In [34]:
help(tree.query_radius)

Help on built-in function query_radius:

query_radius(...) method of sklearn.neighbors._kd_tree.KDTree instance
    query_radius(X, r, return_distance=False,
    count_only=False, sort_results=False)
    
    query the tree for neighbors within a radius r
    
    Parameters
    ----------
    X : array-like of shape (n_samples, n_features)
        An array of points to query
    r : distance within which neighbors are returned
        r can be a single value, or an array of values of shape
        x.shape[:-1] if different radii are desired for each point.
    return_distance : bool, default=False
        if True,  return distances to neighbors of each point
        if False, return only neighbors
        Note that unlike the query() method, setting return_distance=True
        here adds to the computation time.  Not all distances need to be
        calculated explicitly for return_distance=False.  Results are
        not sorted by default: see ``sort_results`` keyword.
    count_only

In [35]:
### **Use Scikit Learn KNN**

In [36]:
X = [[0], [1], [2], [3]]
y = [0, 0, 1, 1]
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X, y)
print(neigh.predict([[1.1]]))
print(neigh.predict_proba([[0.9]]))

[0]
[[0.66666667 0.33333333]]


In [37]:
X, y = zip(*cities)

from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X, y)
print(neigh.predict([[-86, 33]]))
print(neigh.predict_proba([[-86, 33]]))
print(neigh.kneighbors([[-86, 33]], 5, True))

NameError: name 'cities' is not defined

### **State-of-the-Art of MNIST classification**

https://en.wikipedia.org/wiki/MNIST_database (https://en.wikipedia.org/wiki/MNIST_database)
https://rodrigob.github.io/are_we_there_yet/build/classification_datasets_results.html
(https://rodrigob.github.io/are_we_there_yet/build/classification_datasets_results.html)